In [1]:

import ee

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='harare-ndvi-project')

 Define Area of Interest and Time Range

In [ ]:
# Define the area of interest - Harare, Zimbabwe
harare = ee.Geometry.Point(31.0522, -17.8292).buffer(15000)  # 30 km radius

# Set the time range for October 2024
start_date = '2024-10-01'
end_date = '2024-10-31'

Cloud Masking Function

In [ ]:
# Mask clouds using the 'pixel_qa' band
def mask_clouds(image):
    cloud_shadow = 1 << 3
    clouds = 1 << 5
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloud_shadow).eq(0).And(
           qa.bitwiseAnd(clouds).eq(0))
    return image.updateMask(mask)

Load and Prepare Landsat Data

In [ ]:
# Load Landsat 8/9 Surface Reflectance and Thermal data
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(harare) \
    .filterDate(start_date, end_date) \
    .map(mask_clouds) \
    .median()

# Apply scaling factors to optical and thermal bands
def apply_scaling(image):
    optical = image.select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5']) \
        .multiply(0.0000275).add(-0.2)
    thermal = image.select('ST_B10') \
        .multiply(0.00341802).add(149.0)
    return image.addBands(optical, overwrite=True).addBands(thermal, overwrite=True)

landsat = apply_scaling(landsat)

Calculate NDVI and Emissivity

In [ ]:
# Compute NDVI
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Estimate emissivity from NDVI
def calculate_emissivity(ndvi):
    # Proportional vegetation
    pv = ndvi.subtract(0.2).divide(0.3).clamp(0, 1).pow(2)
    # Emissivity formula
    emissivity = pv.multiply(0.004).add(0.986)
    return emissivity.rename('emissivity')

emissivity = calculate_emissivity(ndvi)

Calculate Brightness Temperature and LST

In [ ]:
# Select Brightness Temperature (in Kelvin)
bt = landsat.select('ST_B10').rename('BT')

# Calculate LST using Single Channel Algorithm
def calculate_lst(bt, emissivity):
    wavelength = 10.895  # μm
    p = 14388  # Planck constant * c / σ
    lst = bt.divide(
        ee.Image(1).add(
            wavelength.multiply(bt).divide(emissivity.multiply(p)).log()
        )
    ).subtract(273.15)  # Convert Kelvin to Celsius
    return lst.rename('LST')

lst = calculate_lst(bt, emissivity)

Combine Layers and Visualize

In [ ]:
# Combine bands for export/visualization
final_image = landsat.addBands([ndvi, emissivity, bt, lst])

# Optional visualization using geemap
import geemap
Map = geemap.Map(center=[-17.8292, 31.0522], zoom=10)

lst_vis = {'min': 20, 'max': 45, 'palette': ['blue', 'green', 'yellow', 'orange', 'red']}
Map.addLayer(lst, lst_vis, 'LST (°C) - Oct 2024')
Map.addLayer(ndvi, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'NDVI')
Map.addLayer(emissivity, {'min': 0.97, 'max': 0.99}, 'Emissivity')
Map.addLayerControl()
Map